In [1]:
# Useful Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib
import tensorflow as tf
import tensorflow.keras as keras

### Downloading the Dakshina dataset

In [2]:
#Downloading
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
    
#Uncompressing
!tar -xf dakshina_dataset_v1.0.tar

### Pre processing data

In [3]:
def read(data_path, characters = False):
    
    # Returns the (x, y) pair from the dataset
    # If characters == True, the input/output sample would be in the form list of characters, else as string

    with open(data_path, "r", encoding="utf-8") as f:
        lines = [line.split("\t") for line in f.read().split("\n") if line != '']
    
    x, y = [val[1] for val in lines], [val[0] for val in lines]
    '''if characters:
        input, target = [list(inp_str) for inp_str in input], [list(tar_str) for tar_str in target]'''
    return x, y

In [4]:
START_CHAR = '\t'
END_CHAR = '\n'
BLANK_CHAR = ' '
def encode_decode_characters(train_input, train_target, val_input, val_target):
    
    # Returns the encoding for characters to integer (as a dictionary) and decoding for integers to characters (as a list) for input and target data
    # Encoding and decoding of input vocabulary
    
    input_char_enc = {}
    input_char_dec = []
    max_encoder_seq_length = 1
    for string in train_input + val_input:
        max_encoder_seq_length = max(max_encoder_seq_length, len(string))
        for char in string:
            if char not in input_char_enc:
                input_char_enc[char] = len(input_char_dec)
                input_char_dec.append(char)
    if BLANK_CHAR not in input_char_enc:
        input_char_enc[BLANK_CHAR] = len(input_char_dec)
        input_char_dec.append(BLANK_CHAR)
        
    # Encoding and decoding of target vocabulary
    target_char_enc = {}
    target_char_dec = []
    target_char_enc[START_CHAR] = len(target_char_dec)
    target_char_dec.append(START_CHAR)
    max_decoder_seq_length = 1
    for string in train_target + val_target:
        max_decoder_seq_length = max(max_decoder_seq_length, len(string)+2)
        for char in string:
            if char not in target_char_enc:
                target_char_enc[char] = len(target_char_dec)
                target_char_dec.append(char)
    target_char_enc[END_CHAR] = len(target_char_dec)
    target_char_dec.append(END_CHAR)
    if ' ' not in target_char_enc:
        target_char_enc[BLANK_CHAR] = len(target_char_dec)
        target_char_dec.append(BLANK_CHAR)

    print("Number of training samples:", len(train_input))
    print("Number of validation samples:", len(val_input))
    print("Number of unique input tokens:", len(input_char_dec))
    print("Number of unique output tokens:", len(target_char_dec))
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)

    return input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length

In [5]:
def process_data(input, enc_timesteps, input_char_enc, target = None, dec_timesteps = None, target_char_enc = None):
    
    # Returns the input and target data in a form needed by the Keras embedding layer (i.e) 
    # decoder_input & encoder_input -- (None, timesteps) where each character is encoded by an integer
    # decoder_output -- (None, timesteps, vocabulary size) where the last dimension is the one-hot encoding
    # BLANK_CHAR -- space (equivalent to no meaningful input / blank input)
    
    encoder_input = np.array([[input_char_enc[ch] for ch in string] + [input_char_enc[BLANK_CHAR]] * (enc_timesteps - len(string)) for string in input])

    decoder_input, decoder_target = None, None
    if target is not None and dec_timesteps is not None and target_char_enc is not None:
        
        # START_CHAR -- start of sequence, END_CHAR -- end of sequence
        decoder_input = np.array([[target_char_enc[START_CHAR]] + [target_char_enc[ch] for ch in string] + [target_char_enc[END_CHAR]] 
                                    + [target_char_enc[BLANK_CHAR]] * (dec_timesteps - len(string) - 2) for string in target])
        decoder_target = np.zeros((decoder_input.shape[0], dec_timesteps, len(target_char_enc)), dtype='float32')

        for i in range(decoder_input.shape[0]):
            for t, char_ind in enumerate(decoder_input[i]):
                if t > 0:
                    decoder_target[i,t-1,char_ind] = 1.0
            decoder_target[i,t:,target_char_enc[BLANK_CHAR]] = 1.0

    return encoder_input, decoder_input, decoder_target

In [6]:
train_x, train_y = read('./dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv')
val_x, val_y = read('./dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv')
test_x, test_y = read('./dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv')

# Assigning encoding and decoding for input and target characters
input_char_enc, input_char_dec, target_char_enc, target_char_dec, max_encoder_seq_length, max_decoder_seq_length = encode_decode_characters(
    train_x, train_y, val_x, val_y)

# Assigning training, validation and test encoder input, decoder input, decoder output
train_enc_x, train_dec_x, train_dec_y = process_data(train_x, max_encoder_seq_length, input_char_enc, train_y, 
                                                                  max_decoder_seq_length, target_char_enc)
val_enc_x, val_dec_x, val_dec_y = process_data(val_x, max_encoder_seq_length, input_char_enc, val_y, 
                                                            max_decoder_seq_length, target_char_enc)
test_enc_x, test_dec_x, test_dec_y = process_data(test_x, max_encoder_seq_length, input_char_enc, test_y, 
                                                               max_decoder_seq_length, target_char_enc)

## Seq2Seq Model (without Attention)

### Building the model

In [7]:
def build_model(encoder_vocab_size, decoder_vocab_size, inp_emb_size=64, n_enc_layers=1, n_dec_layers=1, 
                 h_layer_size=64, cell_type='RNN', dropout=0, r_dropout=0, cell_activation='tanh'):
   
    '''
    Function to create a seq2seq model without attention.
    Input :
        encoder_vocab_size -- number of characters in input vocabulary (int)
        decoder_vocab_size -- number of characters in output vocabulary (int)
        inp_emb_size -- size of input embedding layer for encoder and decoder (int, default value : 64)
        n_enc_layers -- number of layers of cell to stack in encoder (int, default value : 1)
        n_dec_layers -- number of layers of cell to stack in decoder (int, default value : 1)
        h_layer_size -- size of hidden layer of the encoder and decoder cells (int, default : 64)
        cell_type -- type of cell used in encoder and decoder (string('LSTM'/ 'GRU'/ 'RNN'), default : 'LSTM')
        dropout -- value of normal dropout (float(between 0 and 1), default : 0.0)
        r_dropout -- value of recurrent dropout (float(between 0 and 1), default : 0.0)
        cell_activation -- type of activation used in the cell (string, default : 'tanh')
    Output :
        model -- (Keras model object)
    '''
    
    # Dictionary of different cell type
    cell_dict = {
        'RNN': keras.layers.SimpleRNN,
        'GRU': keras.layers.GRU,
        'LSTM': keras.layers.LSTM
    }
    
    # Encoder input and embedding
    encoder_input = keras.layers.Input(shape=(None,), name="input_1")
    encoder_inp_emb = keras.layers.Embedding(encoder_vocab_size, inp_emb_size, name="embedding_1")(encoder_input)
    
    # Encoder cell layers
    encoder_seq, *encoder_states = cell_dict[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                      dropout=dropout, recurrent_dropout=r_dropout, name="encoder_1")(encoder_inp_emb)

    for i in range(1, n_enc_layers):
        encoder_seq, *encoder_states = cell_dict[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                                          dropout=dropout, recurrent_dropout=r_dropout, name="encoder_"+str(i+1))(encoder_seq)
    
    
    # Decoder input and embedding
    decoder_input = keras.layers.Input(shape=(None,), name="input_2")
    decoder_inp_emb = keras.layers.Embedding(decoder_vocab_size, inp_emb_size, name="embedding_2")(decoder_input)

    # Decoder cell layers
    decoder_seq, *_ = cell_dict[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                          dropout=dropout, recurrent_dropout=r_dropout, name="decoder_1")(
                                                decoder_inp_emb, initial_state=encoder_states
                                         )
    for i in range(1, n_dec_layers):
        decoder_seq, *_ = cell_dict[cell_type](h_layer_size, activation=cell_activation, return_sequences=True, return_state=True, 
                                              dropout=dropout, recurrent_dropout=r_dropout, name="decoder_"+str(i+1))(
                                                    decoder_seq, initial_state=encoder_states
                                             )
    
    # Softmax Fully Connected dense layer
    decoder_dense_output = keras.layers.Dense(decoder_vocab_size, activation="softmax", name="dense_1")(
        decoder_seq
    )

    # Finally the full encoder-decoder model
    model = keras.Model([encoder_input, decoder_input], decoder_dense_output)

    model.summary(line_length=200)
    return model

In [8]:
build_model(len(input_char_dec), len(target_char_dec))

### Inference Model (without attention)

In [9]:
def create_inference_model(model):
    '''
    Function to return models needed for inference from the original model (without attention).
    Arguments :
        model -- (Keras model object) non-attention model used for training
    Returns :
        encoder_model -- (Keras model object) 
        deocder_model -- (Keras model object)
        no_enc_layers -- (int) number of layers in the encoder
        no_dec_layers -- (int) number of layers in the decoder
    '''
    # Calculating number of layers in encoder and decoder
    no_enc_layers, no_dec_layers = 0, 0
    for layer in model.layers:
        no_enc_layers += layer.name.startswith('encoder')
        no_dec_layers += layer.name.startswith('decoder')

    # Encoder input
    encoder_input = model.input[0]      # Input_1
    # Encoder cell final layer
    encoder_cell = model.get_layer("encoder_"+str(no_enc_layers))
    encoder_type = encoder_cell.__class__.__name__
    encoder_seq, *encoder_state = encoder_cell.output
    # Encoder model
    encoder_model = keras.Model(encoder_input, encoder_state)

    # Decoder input
    decoder_input = model.input[1]      # Input_2
    decoder_inp_emb = model.get_layer("embedding_2")(decoder_input)
    decoder_seq = decoder_inp_emb
    # Inputs to decoder layers' initial states
    decoder_states, decoder_state_inputs = [], []
    for i in range(1, no_dec_layers+1):
        if encoder_type == 'LSTM':
            decoder_state_input = [keras.Input(shape=(encoder_state[0].shape[1],), name="input_"+str(2*i+1)), 
                                   keras.Input(shape=(encoder_state[1].shape[1],), name="input_"+str(2*i+2))]
        else:
            decoder_state_input = [keras.Input(shape=(encoder_state[0].shape[1],), name="input_"+str(i+2))]

        decoder_cell = model.get_layer("decoder_"+str(i))
        decoder_seq, *decoder_state = decoder_cell(decoder_seq, initial_state=decoder_state_input)
        decoder_states += decoder_state
        decoder_state_inputs += decoder_state_input

    # Softmax FC layer
    decoder_dense = model.get_layer("dense_1")
    decoder_dense_output = decoder_dense(decoder_seq)

    # Decoder model
    decoder_model = keras.Model(
        [decoder_input] + decoder_state_inputs, [decoder_dense_output] + decoder_states
    )

    return encoder_model, decoder_model, no_enc_layers, no_dec_layers


def convert_to_word(predictions, char_enc, char_dec = None):
    # Function to return the predictions after cutting the END_CHAR and BLANK_CHAR s at the end.
    # If char_dec == None, the predictions are in the form of decoded string, otherwise as list of integers
    no_samples = len(predictions) if type(predictions) is list else predictions.shape[0]
    pred_words = ['' for _ in range(no_samples)]
    for i, pred_list in enumerate(predictions):
        for l in pred_list:
            # Stop word : END_CHAR
            if l == char_enc[END_CHAR]:
                break
            pred_words[i] += char_dec[l] if char_dec is not None else l
    
    return pred_words